In [1]:
# CELL 1: Setup
import sys
import subprocess

major, minor = sys.version_info[:2]
if not (major == 3 and minor >= 11):
    print(f"❌ Python {major}.{minor} - need 3.11+")
    raise SystemExit
print(f"✅ Python {major}.{minor} OK")

pip_ver = subprocess.check_output(["pip", "--version"]).decode().split()[1]
if not pip_ver.startswith("25"):
    !pip install --upgrade pip==25.*
    print("✅ pip 25 installed. RESTART runtime!")
else:
    print("✅ pip OK")

✅ Python 3.12 OK
✅ pip OK


In [2]:
from qrisp.interface import IQMBackend

# ============================================
# CONFIGURATION - CHANGE THESE AS NEEDED
# ============================================
API_TOKEN = "rkSL242xyAzLHqJQrxHvHs2nREy4AbLE2aQcsA3unuoBnBZTQNV6spN3ItLtE2jJ"
DEVICE = "emerald"  # Changed from sirius to garnet to emerald!

quantum_computer = IQMBackend(
    api_token = API_TOKEN,
    device_instance = DEVICE
)

print(f"✅ Connected to IQM {DEVICE.upper()}!")

/home/amaciejunes/Desktop/IQM/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Connected to IQM EMERALD!


In [3]:
from iqm.qiskit_iqm import IQMProvider

provider = IQMProvider("https://resonance.meetiqm.com",
                       quantum_computer="emerald",
                       token=API_TOKEN)
backend = provider.get_backend()

In [4]:
from qiskit import QuantumCircuit, transpile
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx
from collections import deque

In [5]:
# CELL 2: CALIBRATION DATA

# ========== QUBIT READOUT ERRORS ==========
READOUT_P10 = {  # P(measure 1 | true 0)
    'QB1': 0.0125, 'QB2': 0.0060, 'QB3': 0.0100, 'QB4': 0.0120, 'QB5': 0.0125,
    'QB6': 0.0100, 'QB7': 0.0405, 'QB8': 0.0065, 'QB9': 0.0185, 'QB10': 0.0170,
    'QB11': 0.0095, 'QB12': 0.0140, 'QB13': 0.0180, 'QB14': 0.0160, 'QB15': 0.0125,
    'QB16': 0.0040, 'QB17': 0.0115, 'QB18': 0.0105, 'QB19': 0.0165, 'QB20': 0.0225,
    'QB21': 0.0100, 'QB22': 0.0205, 'QB23': 0.0065, 'QB24': 0.0135, 'QB25': 0.0195,
    'QB26': 0.0080, 'QB27': 0.0085, 'QB28': 0.0105, 'QB29': 0.0280, 'QB30': 0.0175,
    'QB31': 0.0730, 'QB32': 0.0220, 'QB33': 0.0065, 'QB34': 0.0090, 'QB35': 0.0210,
    'QB36': 0.0115, 'QB37': 0.0145, 'QB38': 0.0170, 'QB39': 0.0245, 'QB40': 0.0195,
    'QB41': 0.0185, 'QB42': 0.0235, 'QB43': 0.0165, 'QB44': 0.0150, 'QB45': 0.0290,
    'QB46': 0.0295, 'QB47': 0.0215, 'QB48': 0.0145, 'QB49': 0.0150, 'QB50': 0.0195,
    'QB51': 0.0180, 'QB52': 0.0185, 'QB53': 0.0145, 'QB54': 0.0175
}

READOUT_P01 = {  # P(measure 0 | true 1)
    'QB1': 0.0230, 'QB2': 0.0190, 'QB3': 0.0240, 'QB4': 0.1585, 'QB5': 0.0425,
    'QB6': 0.0105, 'QB7': 0.0505, 'QB8': 0.0160, 'QB9': 0.0235, 'QB10': 0.0275,
    'QB11': 0.0230, 'QB12': 0.0255, 'QB13': 0.0190, 'QB14': 0.0210, 'QB15': 0.0290,
    'QB16': 0.0135, 'QB17': 0.0275, 'QB18': 0.0115, 'QB19': 0.0205, 'QB20': 0.0200,
    'QB21': 0.0155, 'QB22': 0.0175, 'QB23': 0.0130, 'QB24': 0.0260, 'QB25': 0.0220,
    'QB26': 0.0195, 'QB27': 0.0190, 'QB28': 0.0730, 'QB29': 0.0690, 'QB30': 0.0290,
    'QB31': 0.0400, 'QB32': 0.0240, 'QB33': 0.0220, 'QB34': 0.0155, 'QB35': 0.0300,
    'QB36': 0.0135, 'QB37': 0.0400, 'QB38': 0.0255, 'QB39': 0.0270, 'QB40': 0.0395,
    'QB41': 0.0175, 'QB42': 0.0320, 'QB43': 0.0145, 'QB44': 0.0985, 'QB45': 0.0195,
    'QB46': 0.0315, 'QB47': 0.0445, 'QB48': 0.0200, 'QB49': 0.0160, 'QB50': 0.0205,
    'QB51': 0.0815, 'QB52': 0.0315, 'QB53': 0.0250, 'QB54': 0.0625
}

# ========== 2-QUBIT CZ GATE FIDELITIES ==========
# ЭТО КРИТИЧЕСКИ ВАЖНО! Раньше не использовалось!
CZ_FIDELITY = {
    # Best gates (>99%)
    'QB45__QB51': 0.9991,
    'QB31__QB39': 0.9985,
    'QB9__QB3': 0.9982,
    'QB11__QB19': 0.9982,
    'QB41__QB42': 0.9981,
    'QB53__QB49': 0.9980,
    'QB23__QB15': 0.9979,
    'QB29__QB28': 0.9978,
    'QB11__QB12': 0.9977,
    'QB1__QB2': 0.9977,
    'QB20__QB19': 0.9977,
    'QB34__QB33': 0.9975,
    'QB48__QB49': 0.9974,
    'QB18__QB19': 0.9971,
    'QB41__QB40': 0.9970,
    'QB36__QB37': 0.9969,
    'QB13__QB14': 0.9969,
    'QB42__QB43': 0.9969,
    'QB20__QB21': 0.9968,
    'QB48__QB47': 0.9967,
    'QB33__QB41': 0.9966,
    'QB12__QB13': 0.9965,
    'QB32__QB33': 0.9964,
    'QB26__QB27': 0.9963,
    'QB25__QB26': 0.9962,
    'QB17__QB18': 0.9961,
    'QB3__QB4': 0.9960,
    'QB16__QB17': 0.9958,
    'QB19__QB27': 0.9957,
    'QB10__QB11': 0.9955,
    'QB24__QB25': 0.9953,
    'QB2__QB3': 0.9952,
    'QB35__QB43': 0.9951,
    'QB15__QB16': 0.9950,
    'QB49__QB50': 0.9948,
    'QB23__QB24': 0.9946,
    'QB32__QB40': 0.9945,
    'QB34__QB42': 0.9943,
    'QB14__QB22': 0.9941,
    'QB50__QB51': 0.9939,
    'QB9__QB10': 0.9937,
    'QB38__QB39': 0.9935,
    'QB5__QB6': 0.9933,
    'QB1__QB9': 0.9931,
    'QB6__QB14': 0.9929,
    'QB35__QB36': 0.9927,
    'QB37__QB45': 0.9925,
    'QB8__QB16': 0.9920,
    'QB53__QB54': 0.9908,
    'QB45__QB46': 0.9905,
    'QB36__QB44': 0.9903,
    'QB38__QB46': 0.9900,
    'QB4__QB5': 0.9895,
    'QB37__QB38': 0.9890,
    'QB39__QB47': 0.9885,
    'QB46__QB47': 0.9880,
    'QB40__QB48': 0.9875,
    'QB7__QB8': 0.9870,
    'QB27__QB35': 0.9870,
    'QB6__QB7': 0.9865,
    'QB8__QB9': 0.9860,
    'QB43__QB51': 0.9855,
    'QB51__QB52': 0.9850,
    'QB22__QB23': 0.9845,
    'QB14__QB15': 0.9840,
    'QB7__QB15': 0.9835,
    'QB5__QB13': 0.9830,
    'QB13__QB21': 0.9825,
    'QB21__QB29': 0.9820,
    'QB2__QB10': 0.9815,
    'QB10__QB18': 0.9810,
    'QB4__QB12': 0.9805,
    'QB12__QB20': 0.9800,
    # Worse gates (<97%) - AVOID!
    'QB22__QB14': 0.9769,
    'QB28__QB36': 0.9731,
    'QB30__QB31': 0.9726,
    'QB27__QB28': 0.9687,
    'QB52__QB53': 0.9648,
    'QB21__QB22': 0.9622,
    'QB22__QB30': 0.9617,
    'QB21__QB29': 0.9615,
    'QB29__QB21': 0.9601,
    'QB44__QB50': 0.9256,  # ← ОЧЕНЬ ПЛОХОЙ!
}

def get_cz_fidelity(qb1, qb2):
    """Get CZ fidelity between two qubits (order-independent)."""
    key1 = f"{qb1}__{qb2}"
    key2 = f"{qb2}__{qb1}"
    return CZ_FIDELITY.get(key1, CZ_FIDELITY.get(key2, None))

print(f"✅ Calibration data loaded")
print(f"   {len(READOUT_P10)} qubits")
print(f"   {len(CZ_FIDELITY)} CZ gate fidelities")

✅ Calibration data loaded
   54 qubits
   82 CZ gate fidelities


In [6]:
# CELL 3: Configuration

N_QUBITS = 25
SHOTS = 4000
MAX_QUBIT_ERROR = 0.06     # Исключить кубиты с readout error > 5%
MIN_GATE_FIDELITY = 0.98  # ← НОВОЕ! Исключить гейты с fidelity < 98%

print(f"Configuration:")
print(f"   N_QUBITS = {N_QUBITS}")
print(f"   SHOTS = {SHOTS}")
print(f"   MAX_QUBIT_ERROR = {MAX_QUBIT_ERROR}")
print(f"   MIN_GATE_FIDELITY = {MIN_GATE_FIDELITY}  ← НОВОЕ!")

Configuration:
   N_QUBITS = 25
   SHOTS = 4000
   MAX_QUBIT_ERROR = 0.06
   MIN_GATE_FIDELITY = 0.98  ← НОВОЕ!


In [7]:
# CELL 4: Rank qubits and gates

def rank_qubits():
    """Rank qubits by readout error."""
    quality = []
    for qb in READOUT_P10.keys():
        total_error = READOUT_P10[qb] + READOUT_P01[qb]
        quality.append((qb, total_error))
    quality.sort(key=lambda x: x[1])
    return quality

def rank_gates():
    """Rank gates by fidelity."""
    gates = [(pair, fid) for pair, fid in CZ_FIDELITY.items()]
    gates.sort(key=lambda x: x[1], reverse=True)
    return gates

# Show rankings
print("TOP 15 QUBITS (by readout error):")
print("="*50)
for i, (qb, err) in enumerate(rank_qubits()[:15]):
    status = '✅' if err < MAX_QUBIT_ERROR else '⚠️'
    print(f"{i+1:2}. {qb}: error = {err:.4f} {status}")

print(f"\nTOP 10 CZ GATES (by fidelity):")
print("="*50)
for i, (pair, fid) in enumerate(rank_gates()[:10]):
    status = '✅' if fid >= MIN_GATE_FIDELITY else '⚠️'
    print(f"{i+1:2}. {pair}: fidelity = {fid:.4f} {status}")

print(f"\nWORST 5 CZ GATES (EXCLUDED):")
print("="*50)
for pair, fid in rank_gates()[-5:]:
    print(f"   {pair}: fidelity = {fid:.4f} ❌")

TOP 15 QUBITS (by readout error):
 1. QB16: error = 0.0175 ✅
 2. QB23: error = 0.0195 ✅
 3. QB6: error = 0.0205 ✅
 4. QB18: error = 0.0220 ✅
 5. QB8: error = 0.0225 ✅
 6. QB34: error = 0.0245 ✅
 7. QB2: error = 0.0250 ✅
 8. QB36: error = 0.0250 ✅
 9. QB21: error = 0.0255 ✅
10. QB26: error = 0.0275 ✅
11. QB27: error = 0.0275 ✅
12. QB33: error = 0.0285 ✅
13. QB43: error = 0.0310 ✅
14. QB49: error = 0.0310 ✅
15. QB11: error = 0.0325 ✅

TOP 10 CZ GATES (by fidelity):
 1. QB45__QB51: fidelity = 0.9991 ✅
 2. QB31__QB39: fidelity = 0.9985 ✅
 3. QB9__QB3: fidelity = 0.9982 ✅
 4. QB11__QB19: fidelity = 0.9982 ✅
 5. QB41__QB42: fidelity = 0.9981 ✅
 6. QB53__QB49: fidelity = 0.9980 ✅
 7. QB23__QB15: fidelity = 0.9979 ✅
 8. QB29__QB28: fidelity = 0.9978 ✅
 9. QB11__QB12: fidelity = 0.9977 ✅
10. QB1__QB2: fidelity = 0.9977 ✅

WORST 5 CZ GATES (EXCLUDED):
   QB21__QB22: fidelity = 0.9622 ❌
   QB22__QB30: fidelity = 0.9617 ❌
   QB21__QB29: fidelity = 0.9615 ❌
   QB29__QB21: fidelity = 0.9601 ❌
   QB4

In [8]:
# CELL 5: Build weighted graph (FIXED!)

def build_weighted_graph(backend, max_qubit_error, min_gate_fidelity):
    """
    Build graph with:
    - Nodes: Only good qubits (error < max_qubit_error)
    - Edges: Only good gates (fidelity >= min_gate_fidelity)
    - Node weight: qubit quality (1 - error)
    - Edge weight: gate fidelity
    """
    G = nx.Graph()

    # Add nodes (only good qubits)
    good_qubits = []
    bad_qubits = []

    for qb in READOUT_P10.keys():
        idx = int(qb[2:]) - 1  # QB1 -> 0
        error = READOUT_P10[qb] + READOUT_P01[qb]

        if error < max_qubit_error:
            G.add_node(idx,
                       name=qb,
                       error=error,
                       quality=1-error,
                       p10=READOUT_P10[qb],
                       p01=READOUT_P01[qb])
            good_qubits.append(qb)
        else:
            bad_qubits.append((qb, error))

    print(f"Nodes (good qubits): {G.number_of_nodes()}")
    print(f"Excluded qubits: {[q for q,e in bad_qubits]}")

    # Add edges (only good gates between good qubits)
    good_edges = 0
    bad_edges = 0
    missing_edges = 0

    for edge in backend.coupling_map:
        idx1, idx2 = edge
        qb1 = f"QB{idx1+1}"
        qb2 = f"QB{idx2+1}"

        # Skip if either qubit is bad
        if idx1 not in G.nodes or idx2 not in G.nodes:
            continue

        # Get CZ fidelity
        fidelity = get_cz_fidelity(qb1, qb2)

        if fidelity is None:
            missing_edges += 1
            # Use default if no calibration data
            fidelity = 0.98

        # Only add edge if fidelity is good enough
        if fidelity >= min_gate_fidelity:
            if not G.has_edge(idx1, idx2):  # Avoid duplicates
                G.add_edge(idx1, idx2, fidelity=fidelity)
                good_edges += 1
        else:
            bad_edges += 1

    print(f"\nEdges (good gates): {G.number_of_edges()}")
    print(f"Excluded gates (fidelity < {min_gate_fidelity}): {bad_edges}")
    if missing_edges > 0:
        print(f"Gates with missing calibration data: {missing_edges}")

    return G

# Build graph
G = build_weighted_graph(backend, MAX_QUBIT_ERROR, MIN_GATE_FIDELITY)

print(f"\n✅ Graph built: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

Nodes (good qubits): 44
Excluded qubits: ['QB4', 'QB7', 'QB28', 'QB29', 'QB31', 'QB44', 'QB46', 'QB47', 'QB51', 'QB54']

Edges (good gates): 55
Excluded gates (fidelity < 0.98): 4
Gates with missing calibration data: 14

✅ Graph built: 44 nodes, 55 edges


In [9]:
# CELL 6: Find optimal path (FIXED!)

def find_optimal_path(G, n_qubits):
    """
    Find optimal connected path maximizing:
    - Sum of qubit qualities (node weights)
    - Sum of gate fidelities (edge weights)

    Path score = Σ(1 - qubit_error) + 2 * Σ(gate_fidelity)

    Edge weights are doubled because GHZ needs N-1 CX gates!
    """

    if G.number_of_nodes() < n_qubits:
        print(f"❌ Not enough good qubits: {G.number_of_nodes()} < {n_qubits}")
        return None

    def path_score(path):
        """Calculate total score for a path."""
        if len(path) < 2:
            return 0

        # Node score (qubit quality)
        node_score = sum(G.nodes[n]['quality'] for n in path)

        # Edge score (gate fidelity) - WEIGHTED 2x!
        edge_score = 0
        for i in range(len(path) - 1):
            if G.has_edge(path[i], path[i+1]):
                edge_score += G.edges[path[i], path[i+1]]['fidelity']
            else:
                return -1000  # Invalid path!

        return node_score + 2 * edge_score

    def dfs_best_path(start, n):
        """DFS to find best scoring path."""
        best_path = [start]
        best_score = 0

        stack = [(start, [start])]

        while stack:
            curr, path = stack.pop()

            if len(path) == n:
                score = path_score(path)
                if score > best_score:
                    best_score = score
                    best_path = path.copy()
                continue

            if len(path) > len(best_path):
                score = path_score(path)
                if score > best_score:
                    best_score = score
                    best_path = path.copy()

            # Get neighbors sorted by combined score
            neighbors = []
            for nb in G.neighbors(curr):
                if nb not in path:
                    nb_quality = G.nodes[nb]['quality']
                    edge_fidelity = G.edges[curr, nb]['fidelity']
                    combined = nb_quality + edge_fidelity
                    neighbors.append((nb, combined))

            neighbors.sort(key=lambda x: x[1], reverse=True)

            for nb, _ in neighbors[:5]:  # Explore top 5
                stack.append((nb, path + [nb]))

        return best_path, best_score

    # Try starting from best qubits
    sorted_nodes = sorted(G.nodes(), key=lambda n: G.nodes[n]['quality'], reverse=True)

    overall_best = []
    overall_best_score = -1

    print(f"Searching for {n_qubits}-qubit path...")

    for i, start in enumerate(sorted_nodes[:15]):
        path, score = dfs_best_path(start, n_qubits)
        if len(path) >= n_qubits and score > overall_best_score:
            overall_best = path[:n_qubits]
            overall_best_score = score
            print(f"   Found path from QB{start+1}: score = {score:.4f}")

    if len(overall_best) < n_qubits:
        print(f"\n⚠️ Could only find path of {len(overall_best)} qubits")
        return overall_best if overall_best else None

    return overall_best

# Find optimal path
optimal_path = find_optimal_path(G, N_QUBITS)

if optimal_path:
    path_names = [f'QB{i+1}' for i in optimal_path]

    # Analyze path quality
    path_errors = [READOUT_P10[qb] + READOUT_P01[qb] for qb in path_names]
    path_fidelities = []
    for i in range(len(optimal_path) - 1):
        qb1 = f'QB{optimal_path[i]+1}'
        qb2 = f'QB{optimal_path[i+1]+1}'
        fid = get_cz_fidelity(qb1, qb2)
        path_fidelities.append(fid if fid else 0.98)

    print(f"\n{'='*60}")
    print(f"OPTIMAL PATH FOUND!")
    print(f"{'='*60}")
    print(f"Qubits: {path_names}")
    print(f"\nQubit metrics:")
    print(f"   Avg readout error: {np.mean(path_errors):.4f}")
    print(f"   Max readout error: {np.max(path_errors):.4f}")
    print(f"\nGate metrics:")
    print(f"   Avg CZ fidelity:   {np.mean(path_fidelities):.4f}")
    print(f"   Min CZ fidelity:   {np.min(path_fidelities):.4f}")
    print(f"\nExpected measurement success:")
    p10 = np.array([READOUT_P10[qb] for qb in path_names])
    p01 = np.array([READOUT_P01[qb] for qb in path_names])
    print(f"   P(meas=00|true=00): {np.prod(1-p10):.4f}")
    print(f"   P(meas=11|true=11): {np.prod(1-p01):.4f}")
else:
    print("❌ Could not find valid path!")

Searching for 25-qubit path...
   Found path from QB16: score = 71.7832
   Found path from QB23: score = 71.8122
   Found path from QB6: score = 71.8368
   Found path from QB18: score = 71.8424
   Found path from QB8: score = 71.8662

OPTIMAL PATH FOUND!
Qubits: ['QB8', 'QB9', 'QB10', 'QB11', 'QB12', 'QB13', 'QB21', 'QB20', 'QB19', 'QB18', 'QB17', 'QB16', 'QB15', 'QB23', 'QB24', 'QB25', 'QB26', 'QB27', 'QB35', 'QB43', 'QB42', 'QB34', 'QB33', 'QB41', 'QB40']

Qubit metrics:
   Avg readout error: 0.0354
   Max readout error: 0.0590

Gate metrics:
   Avg CZ fidelity:   0.9948
   Min CZ fidelity:   0.9825

Expected measurement success:
   P(meas=00|true=00): 0.7088
   P(meas=11|true=11): 0.5775


In [10]:
# CELL 7: Mitigation functions

def mitigate_zz(counts, n, p10, p01):
    """Mitigate ZZ expectation value."""
    total = sum(counts.values())
    all_0 = '0' * n
    all_1 = '1' * n

    p_00_00 = np.prod(1 - p10)
    p_11_11 = np.prod(1 - p01)
    p_00_11 = np.prod(p01)
    p_11_00 = np.prod(p10)

    A = np.array([[p_00_00, p_00_11], [p_11_00, p_11_11]])
    p_meas = np.array([counts.get(all_0, 0)/total, counts.get(all_1, 0)/total])

    try:
        A_inv = np.linalg.inv(A)
        p_true = np.clip(A_inv @ p_meas, 0, 1)
        if p_true.sum() > 1:
            p_true /= p_true.sum()
        return 2 * (p_true[0] + p_true[1]) - 1, p_true[0], p_true[1]
    except:
        p00 = counts.get(all_0, 0) / total
        p11 = counts.get(all_1, 0) / total
        return 2*(p00+p11)-1, p00, p11

def mitigate_xx(counts, n, p10, p01):
    """Mitigate XX expectation value."""
    total = sum(counts.values())
    p_flip = (p10 + p01) / 2
    parity_factor = np.prod(1 - 2 * p_flip)

    xx_raw = sum((1 if s.count('1')%2==0 else -1) * c/total for s,c in counts.items())

    if abs(parity_factor) > 0.01:
        xx_mit = np.clip(xx_raw / parity_factor, -1, 1)
    else:
        xx_mit = xx_raw

    return xx_mit, xx_raw

def create_ghz(n, basis='Z'):
    qc = QuantumCircuit(n, n)
    qc.h(0)
    for i in range(n - 1):
        qc.cx(i, i + 1)
    if basis == 'X':
        for i in range(n):
            qc.h(i)
    qc.measure_all()
    return qc

print("✅ Mitigation functions ready")

✅ Mitigation functions ready


In [11]:
# CELL 8: Create circuits

if optimal_path:
    qc_z = create_ghz(N_QUBITS, 'Z')
    qc_x = create_ghz(N_QUBITS, 'X')

    qc_z_t = transpile(qc_z, backend, initial_layout=optimal_path, optimization_level=3)
    qc_x_t = transpile(qc_x, backend, initial_layout=optimal_path, optimization_level=3)

    print(f"Circuits created with optimal path")
    print(f"   Depth: {qc_z_t.depth()}")
    print(f"   Gates: {sum(qc_z_t.count_ops().values())}")
else:
    print("❌ No valid path found!")

Circuits created with optimal path
   Depth: 50
   Gates: 99


---
## ⚠️ EXPERIMENT (costs credits!)
---

In [16]:
from iqm.iqm_client import CircuitCompilationOptions
from iqm.iqm_client.models import DDMode, STANDARD_DD_STRATEGY, HeraldingMode
# CELL 9: Run experiment

print(f"{'='*60}")
print(f"GHZ EXPERIMENT: {N_QUBITS} qubits, {SHOTS} shots")
print(f"{'='*60}")


cco = CircuitCompilationOptions(
    dd_mode=DDMode.ENABLED,
    dd_strategy=STANDARD_DD_STRATEGY,
    heralding_mode=HeraldingMode.ZEROS,
)
print("\n[1/2] Z-basis...")
job_z = backend.run(qc_z_t, shots=SHOTS, circuit_compilation_options=cco)
print(f"      Job: {job_z.job_id()}")
counts_z = job_z.result().get_counts()

print("[2/2] X-basis...")
job_x = backend.run(qc_x_t, shots=SHOTS)
print(f"      Job: {job_x.job_id()}")
counts_x = job_x.result().get_counts()

print("\n✅ Done!")

GHZ EXPERIMENT: 25 qubits, 4000 shots

[1/2] Z-basis...
      Job: 019c177e-fdfe-7332-b456-b6c806fe14ce


Progress in queue:   0%|          | 0/1 [00:09<?, ?it/s]


[2/2] X-basis...
      Job: 019c177f-306a-75d3-a217-4cac88990662


Progress in queue:   0%|          | 0/2 [00:06<?, ?it/s]



✅ Done!


In [17]:
# CELL 10: Results

print(f"{'='*60}")
print("RESULTS")
print(f"{'='*60}")

path_names = [f'QB{i+1}' for i in optimal_path]
p10 = np.array([READOUT_P10[qb] for qb in path_names])
p01 = np.array([READOUT_P01[qb] for qb in path_names])

total_z = sum(counts_z.values())
total_x = sum(counts_x.values())

# RAW
p00_raw = counts_z.get('0'*N_QUBITS, 0) / total_z
p11_raw = counts_z.get('1'*N_QUBITS, 0) / total_z
zz_raw = 2 * (p00_raw + p11_raw) - 1
xx_raw = sum((1 if s.count('1')%2==0 else -1) * c/total_x for s,c in counts_x.items())
f_raw = 0.5 + 0.25 * (zz_raw + xx_raw)

# MITIGATED
zz_mit, p00_mit, p11_mit = mitigate_zz(counts_z, N_QUBITS, p10, p01)
xx_mit, _ = mitigate_xx(counts_x, N_QUBITS, p10, p01)
f_mit = 0.5 + 0.25 * (zz_mit + xx_mit)

print(f"\nPath: {path_names}")
print(f"\n{'Metric':<12} | {'RAW':>10} | {'MITIGATED':>10} | {'Change':>10}")
print("-" * 50)
print(f"{'P(00...0)':<12} | {p00_raw:>10.4f} | {p00_mit:>10.4f} | {p00_mit-p00_raw:>+10.4f}")
print(f"{'P(11...1)':<12} | {p11_raw:>10.4f} | {p11_mit:>10.4f} | {p11_mit-p11_raw:>+10.4f}")
print(f"{'⟨ZZ...Z⟩':<12} | {zz_raw:>10.4f} | {zz_mit:>10.4f} | {zz_mit-zz_raw:>+10.4f}")
print(f"{'⟨XX...X⟩':<12} | {xx_raw:>10.4f} | {xx_mit:>10.4f} | {xx_mit-xx_raw:>+10.4f}")
print("-" * 50)
print(f"{'FIDELITY':<12} | {f_raw:>10.4f} | {f_mit:>10.4f} | {f_mit-f_raw:>+10.4f}")

print(f"\n{'='*60}")
print("ENTANGLEMENT VERIFICATION")
print(f"{'='*60}")

print(f"\nRAW:       F = {f_raw:.4f}  {'✅ PROVEN!' if f_raw > 0.5 else '❌'}")
print(f"MITIGATED: F = {f_mit:.4f}  {'✅ PROVEN!' if f_mit > 0.5 else '❌'}")

if f_mit > 0.5:
    print(f"\n🎉 {N_QUBITS}-QUBIT ENTANGLEMENT PROVEN!")
else:
    print(f"\n💡 Try reducing N_QUBITS to {N_QUBITS - 2}")

RESULTS

Path: ['QB8', 'QB9', 'QB10', 'QB11', 'QB12', 'QB13', 'QB21', 'QB20', 'QB19', 'QB18', 'QB17', 'QB16', 'QB15', 'QB23', 'QB24', 'QB25', 'QB26', 'QB27', 'QB35', 'QB43', 'QB42', 'QB34', 'QB33', 'QB41', 'QB40']

Metric       |        RAW |  MITIGATED |     Change
--------------------------------------------------
P(00...0)    |     0.0755 |     0.1065 |    +0.0310
P(11...1)    |     0.0185 |     0.0320 |    +0.0135
⟨ZZ...Z⟩     |    -0.8120 |    -0.7229 |    +0.0891
⟨XX...X⟩     |     0.1335 |     0.3289 |    +0.1954
--------------------------------------------------
FIDELITY     |     0.3304 |     0.4015 |    +0.0711

ENTANGLEMENT VERIFICATION

RAW:       F = 0.3304  ❌
MITIGATED: F = 0.4015  ❌

💡 Try reducing N_QUBITS to 23
